In [7]:
!pip install torch torchvision timm

import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from timm import create_model

Defaulting to user installation because normal site-packages is not writeable


In [8]:
from pathlib import Path
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as T

class UECFood100Detection(Dataset):
    def __init__(self, root_dir, bb_info_path, transforms=None):
        self.root = Path(root_dir)
        ann_path = Path(bb_info_path)

        self.samples   = {}
        self.img_paths = {}

        with ann_path.open() as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5 or parts[0].lower() == "img_name":
                    continue

                if len(parts) == 6:
                    cls_str, img_name, *coords = parts
                else:
                    img_name, *coords = parts
                    cls_str = ann_path.parent.name

                try:
                    lbl       = int(cls_str)
                    x1, y1, x2, y2 = map(int, coords)
                except ValueError:
                    continue

                if img_name not in self.img_paths:
                    class_folder = self.root / cls_str
                    candidate = class_folder / img_name
                    if not candidate.exists():
                        matches = list(class_folder.glob(f"{Path(img_name).stem}.*"))
                        if not matches:
                            raise FileNotFoundError(f"No file matches {img_name} in {class_folder}")
                        candidate = matches[0]
                    self.img_paths[img_name] = candidate

                self.samples.setdefault(img_name, []).append(([x1, y1, x2, y2], lbl))

        self.imgs       = sorted(self.samples.keys())
        self.transforms = transforms or T.Compose([T.ToTensor()])

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        img_path = self.img_paths[img_name]

        img = Image.open(img_path).convert("RGB")
        w, h = img.size

        boxes, labels = [], []
        for (x1, y1, x2, y2), lbl in self.samples[img_name]:
            boxes.append([max(0,x1), max(0,y1), min(w,x2), min(h,y2)])
            labels.append(lbl)

        target = {
            "boxes":  torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
            "image_id": torch.tensor([idx]),
        }

        return self.transforms(img), target

In [9]:
BASE_DIR = Path("UECFOOD100") / "UECFOOD100"
all_ann = BASE_DIR / "all_bb_info.txt"

with open(all_ann, "w") as fout:
    for ann in sorted(BASE_DIR.rglob("bb_info.txt")):
        cls = ann.parent.name  
        with open(ann) as fin:
            for line in fin:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                if parts[0] == "img_name" and parts[1] == "x1":
                    continue
                fout.write(f"{cls} {' '.join(parts)}\n")

print(f"Created merged annotation file at {all_ann}")

Created merged annotation file at UECFOOD100/UECFOOD100/all_bb_info.txt


In [11]:
dataset = UECFood100Detection(
    root_dir=str(BASE_DIR),
    bb_info_path=str(all_ann)
)
print("Total images with boxes:", len(dataset))

Total images with boxes: 12740


In [12]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tuple(zip(*batch))

idxs = list(range(len(dataset)))
train_idxs, val_idxs = train_test_split(idxs, test_size=0.2, random_state=42)

train_ds = torch.utils.data.Subset(dataset, train_idxs)
val_ds   = torch.utils.data.Subset(dataset, val_idxs)

train_loader = DataLoader(
    train_ds,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)
val_loader = DataLoader(
    val_ds,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

print(f"Train images: {len(train_ds)}, Val images: {len(val_ds)}")

Train images: 10192, Val images: 2548


In [15]:
#Baseline Faster R‑CNN (ResNet‑50 FPN)

from torchvision.models import ResNet50_Weights
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.optim import SGD
from tqdm import tqdm 

baseline_model = fasterrcnn_resnet50_fpn(
    weights=None,
    weights_backbone=ResNet50_Weights.IMAGENET1K_V1,
    num_classes=NUM_CLASSES
).to(device)

baseline_optimizer = SGD(
    baseline_model.parameters(),
    lr=0.005,     
    momentum=0.9,
    weight_decay=1e-4
)

def train_one_epoch(model, optimizer, data_loader, device, print_freq=100):
    model.train()
    for i, (images, targets) in enumerate(tqdm(data_loader, desc="Baseline Train"), 1):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % print_freq == 0:
            print(f"  iter {i}/{len(data_loader)} — loss: {loss.item():.4f}") 

def evaluate_loss(model, data_loader, device):
    model.train()
    losses = []
    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Baseline Val Loss"):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            total_loss = sum(loss for loss in loss_dict.values()).item()
            losses.append(total_loss)
    model.eval()
    return sum(losses) / len(losses)

num_epochs_baseline = 3
for epoch in range(1, num_epochs_baseline + 1):
    print(f"\n[Baseline] Epoch {epoch}/{num_epochs_baseline}")
    train_one_epoch(baseline_model, baseline_optimizer, train_loader, device)
    val_loss = evaluate_loss(baseline_model, val_loader, device)
    print(f"[Baseline] Validation Loss: {val_loss:.4f}")


[Baseline] Epoch 1/3


Baseline Train:   4%|▍         | 101/2548 [00:37<15:17,  2.67it/s]

  iter 100/2548 — loss: 0.3401


Baseline Train:   8%|▊         | 200/2548 [01:15<17:23,  2.25it/s]

  iter 200/2548 — loss: 0.2911


...

  iter 2400/2548 — loss: 0.2013


Baseline Train:  98%|█████████▊| 2500/2548 [16:01<00:19,  2.44it/s]

  iter 2500/2548 — loss: 0.1828


Baseline Val Loss: 100%|██████████| 637/637 [01:59<00:00,  5.32it/s]

[Baseline] Validation Loss: 0.2448


In [18]:
from collections import OrderedDict
import torch.nn as nn

raw_backbone = create_model(
    'efficientnet_b0',
    pretrained=True,
    features_only=True,
    out_indices=(4,)   
)

class EfficientNetBackbone(nn.Module):
    def __init__(self, timm_backbone):
        super().__init__()
        self.body = timm_backbone
        self.out_channels = self.body.feature_info.channels()[-1] 

    def forward(self, x):
        feats = self.body(x)               
        return OrderedDict([("0", feats[0])])

backbone = EfficientNetBackbone(raw_backbone)

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


In [20]:
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN

anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

model = FasterRCNN(
    backbone,
    num_classes=101,
    rpn_anchor_generator=anchor_generator
).cuda()

In [21]:
model.train()
img, target = dataset[0]
loss_dict = model([img.cuda()], [{k: v.cuda() for k,v in target.items()}])
print({k: float(v) for k,v in loss_dict.items()})

{'loss_classifier': 4.770044326782227, 'loss_box_reg': 0.2090047001838684, 'loss_objectness': 0.6847230792045593, 'loss_rpn_box_reg': 0.04593349248170853}


In [22]:
print(f" train_loader batches: {len(train_loader)}")
print(f" val_loader   batches: {len(val_loader)}")

for epoch in range(1, 2):   
    print(f"--- Starting epoch {epoch} ---")
    model.train()
    for batch_idx, (imgs, targets) in enumerate(train_loader):
        if batch_idx == 0:
            print(f"  Got batch 0: {len(imgs)} images, {len(targets)} target dicts")
        break
    model.eval()
    for batch_idx, (imgs, targets) in enumerate(val_loader):
        if batch_idx == 0:
            print(f"  Got val batch 0: {len(imgs)} images, {len(targets)} target dicts")
        break
print("Debug done.")

 train_loader batches: 2548
 val_loader   batches: 637
--- Starting epoch 1 ---
  Got batch 0: 4 images, 4 target dicts
  Got val batch 0: 4 images, 4 target dicts
Debug done.


In [33]:
num_epochs = 10

for epoch in range(1, num_epochs + 1):
    print(f"Starting Epoch {epoch}/{num_epochs}")

    model.train()
    total_train_loss = 0.0
    for batch_idx, (imgs, targets) in enumerate(train_loader, 1):
        imgs    = [img.cuda() for img in imgs]
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        loss_dict = model(imgs, targets)
        loss = sum(loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        if batch_idx % 500 == 0:
            print(f"   • Batch {batch_idx}/{len(train_loader)} — curr loss: {loss.item():.4f}")

    lr_scheduler.step()
    avg_train = total_train_loss / len(train_loader)

    total_val_loss = 0.0
    with torch.no_grad():
        model.train()  
        for imgs, targets in val_loader:
            imgs    = [img.cuda() for img in imgs]
            targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

            loss_dict = model(imgs, targets)
            total_val_loss += sum(loss_dict.values()).item()

    avg_val = total_val_loss / len(val_loader)
    print(f" Epoch {epoch} complete — Train loss: {avg_train:.6f} — Val loss: {avg_val:.6f}\n")

Starting Epoch 1/10
   • Batch 500/2548 — curr loss: 5.4668
   • Batch 1000/2548 — curr loss: 5.5973
   • Batch 1500/2548 — curr loss: 5.7372
   • Batch 2000/2548 — curr loss: 5.6331
   • Batch 2500/2548 — curr loss: 5.6053
 Epoch 1 complete — Train loss: 5.636133 — Val loss: 5.632444

Starting Epoch 2/10
   • Batch 500/2548 — curr loss: 5.8262
   • Batch 1000/2548 — curr loss: 5.5411
   • Batch 1500/2548 — curr loss: 5.5906
   • Batch 2000/2548 — curr loss: 5.4399
   • Batch 2500/2548 — curr loss: 5.6176
 Epoch 2 complete — Train loss: 5.636046 — Val loss: 5.631989

Starting Epoch 3/10
   • Batch 500/2548 — curr loss: 5.6407
   • Batch 1000/2548 — curr loss: 5.7055
   • Batch 1500/2548 — curr loss: 5.6234
   • Batch 2000/2548 — curr loss: 5.5711
   • Batch 2500/2548 — curr loss: 5.7668
 Epoch 3 complete — Train loss: 5.635589 — Val loss: 5.632226

Starting Epoch 4/10
   • Batch 500/2548 — curr loss: 5.6427
   • Batch 1000/2548 — curr loss: 5.7461
   • Batch 1500/2548 — curr loss: 5.75

In [27]:
#Hyperparameter Tuning
from collections import OrderedDict
import torch.nn as nn

class EfficientNetBackbone(nn.Module):
    def __init__(self, raw_backbone):
        super().__init__()
        self.body = raw_backbone
        self.out_channels = raw_backbone.feature_info.info[-1]['num_chs']

    def forward(self, x):
        feats = self.body(x)
        return OrderedDict([("0", feats[-1])])
    
import itertools
import pandas as pd
from timm import create_model
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import MultiScaleRoIAlign
from torch.optim import SGD

def train_and_eval(lr, wd, num_epochs=5):
    raw = create_model("efficientnet_b0", pretrained=True, features_only=True)
    backbone = EfficientNetBackbone(raw)

    anchor_gen = AnchorGenerator(
        sizes=((32, 64, 128, 256, 512),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )
    roi_pooler = MultiScaleRoIAlign(
        featmap_names=["0"],
        output_size=7,
        sampling_ratio=2
    )
    
    model = FasterRCNN(
        backbone,
        num_classes=NUM_CLASSES,
        rpn_anchor_generator=anchor_gen,
        box_roi_pool=roi_pooler
    ).to(device)
    optim = SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)

    for _ in range(num_epochs):
        train_one_epoch(model, optim, train_loader, device)
    return evaluate_loss(model, val_loader, device)

lrs = [1e-3, 5e-4, 1e-4]
wds = [1e-4, 1e-5]
results = []
for lr, wd in itertools.product(lrs, wds):
    loss = train_and_eval(lr, wd, num_epochs=1)
    print(f"lr={lr:.0e}, wd={wd:.0e} → val_loss={loss:.4f}")
    results.append({"lr": lr, "weight_decay": wd, "val_loss": loss})

df = pd.DataFrame(results).sort_values("val_loss").reset_index(drop=True)
print(df.head())

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
Baseline Train:   4%|▍         | 101/2548 [00:18<07:05,  5.75it/s]

  iter 100/2548 — loss: 0.9036


Baseline Train:   8%|▊         | 201/2548 [00:35<06:26,  6.07it/s]

  iter 200/2548 — loss: 0.6728


Baseline Train:  12%|█▏        | 301/2548 [00:53<06:37,  5.66it/s]

...

  iter 2400/2548 — loss: 0.5418


Baseline Train:  98%|█████████▊| 2501/2548 [07:25<00:08,  5.59it/s]

  iter 2500/2548 — loss: 0.6119


Baseline Val Loss: 100%|██████████| 637/637 [00:43<00:00, 14.62it/s]

lr=1e-04, wd=1e-05 → val_loss=0.6672
       lr  weight_decay  val_loss
0  0.0010       0.00010  0.479602
1  0.0010       0.00001  0.492483
2  0.0005       0.00001  0.545558
3  0.0005       0.00010  0.553480
4  0.0001       0.00001  0.667174


In [28]:
from timm import create_model
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import MultiScaleRoIAlign
from torch.optim import SGD
from collections import OrderedDict
import torch.nn as nn

class EfficientNetBackbone(nn.Module):
    def __init__(self, raw):
        super().__init__()
        self.body = raw
        self.out_channels = raw.feature_info.info[-1]['num_chs']
    def forward(self, x):
        feats = self.body(x)
        return OrderedDict([("0", feats[-1])])

raw = create_model('efficientnet_b0', pretrained=True, features_only=True)
backbone = EfficientNetBackbone(raw)
anchor_gen = AnchorGenerator(sizes=((32,64,128,256,512),), aspect_ratios=((0.5,1.0,2.0),))
roi_pooler = MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)

model = FasterRCNN(
    backbone, 
    num_classes=NUM_CLASSES,
    rpn_anchor_generator=anchor_gen,
    box_roi_pool=roi_pooler
).to(device)

optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4)

num_epochs_final = 2
for epoch in range(1, num_epochs_final + 1):
    print(f"\n— Final Train Epoch {epoch}/{num_epochs_final} —")
    train_one_epoch(model, optimizer, train_loader, device)
    val_loss = evaluate_loss(model, val_loader, device)
    print(f"Validation Loss: {val_loss:.4f}")


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.



— Final Train Epoch 1/2 —


Baseline Train:   4%|▍         | 101/2548 [00:18<07:04,  5.77it/s]

  iter 100/2548 — loss: 0.7562


Baseline Train:   8%|▊         | 201/2548 [00:36<06:47,  5.76it/s]

...

  iter 2400/2548 — loss: 0.5234


Baseline Train:  98%|█████████▊| 2501/2548 [07:42<00:08,  5.80it/s]

  iter 2500/2548 — loss: 0.3807


Baseline Val Loss: 100%|██████████| 637/637 [00:44<00:00, 14.20it/s]

Validation Loss: 0.3860


In [29]:
from torchvision.ops import box_iou

model.eval()
ious = []

with torch.no_grad():
    for images, targets in val_loader:
        images = [img.to(device) for img in images]
        outputs = model(images) 
        for out, tgt in zip(outputs, targets):
            gt_boxes = tgt['boxes'].to(device)
            pred_boxes = out['boxes']

            if len(pred_boxes) == 0 or len(gt_boxes) == 0:
                continue

            iou_mat = box_iou(pred_boxes, gt_boxes)
            best_iou_per_pred, _ = iou_mat.max(dim=1)
            ious += best_iou_per_pred.cpu().tolist()

mean_iou = sum(ious) / len(ious)
print(f"Mean IoU over all predictions: {mean_iou:.3f}")

Mean IoU over all predictions: 0.559


In [30]:
model = baseline_model
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu